In [30]:
from nltk import data
from nltk.corpus.reader import CHILDESCorpusReader
from nltk.probability import FreqDist
import entropies7.entropies7 as Ent
from numpy import *
import scipy as sp
import numpy as np
import re

%matplotlib inline
from pandas import *
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
nmmfile = "/Users/dspoka/Desktop/moscoso/nltk_childes/NLTKCHILDES/morph-eng.csv"
nmsfile = "/Users/dspoka/Desktop/moscoso/nltk_childes/NLTKCHILDES/syntax-eng.csv"



In [36]:
def get_files(corpus):
    if corpus =='childes':
    #     corpus_root = data.find('corpora/childes/Eng-USA-MOR')
        corpus_root = data.find('corpora/childes/English-UK-MOR')
        eng = CHILDESCorpusReader(corpus_root, 'Manchester/.*.xml')
        nmmfile = "/Users/dspoka/Desktop/moscoso/nltk_childes/NLTKCHILDES/morph-eng.csv"
        nmsfile = "/Users/dspoka/Desktop/moscoso/nltk_childes/NLTKCHILDES/syntax-eng.csv"
#         nmmfile = "/Users/jeremyirvin/Desktop/SeniorThesis/Childes/nltk_childes/NLTKCHILDES/morph-eng.csv"
#         nmsfile = "/Users/jeremyirvin/Desktop/SeniorThesis/Childes/nltk_childes/NLTKCHILDES/syntax-eng.csv"
        files = eng.fileids()
    return eng, files
# %timeit files = get_files('childes')
eng, files = get_files('childes')
# chiles has 804 files

In [37]:
def convert_age_to_days(age):
    parsed_age = re.split('P|Y|M|D',age)
    years = parsed_age[1]
    months = parsed_age[2]
    days = parsed_age[3]
    if days == '':
        days = 0
    return int(years)*365+int(months)*30+int(days)

In [39]:
def compare(f1, f2):
    age1 = convert_age_to_days(eng.age(f1)[0])
    age2 = convert_age_to_days(eng.age(f2)[0])
    if age1 < age2:
        return -1
    elif age2 < age1:
        return 1
    else:
        return 0
   
def group_files_by_child_age(files):
    # Group files by age, child, and corpus
    resDict = {}
    for f in files:
        child = f.split("/")[1]
        if resDict.has_key(child):
            resDict[child] += [f]
        else:
            resDict[child] = [f]
    for child in resDict.keys():
        resDict[child] = sorted(resDict[child], cmp=compare)
    
    return resDict

# %timeit resDict = group_files_by_child_age(files)
resDict = group_files_by_child_age(files)

In [40]:
resDict

{u'anne': [u'Manchester/anne/anne01a.xml',
  u'Manchester/anne/anne01b.xml',
  u'Manchester/anne/anne02a.xml',
  u'Manchester/anne/anne02b.xml',
  u'Manchester/anne/anne03a.xml',
  u'Manchester/anne/anne03b.xml',
  u'Manchester/anne/anne04a.xml',
  u'Manchester/anne/anne04b.xml',
  u'Manchester/anne/anne05a.xml',
  u'Manchester/anne/anne05b.xml',
  u'Manchester/anne/anne06a.xml',
  u'Manchester/anne/anne06b.xml',
  u'Manchester/anne/anne07a.xml',
  u'Manchester/anne/anne07b.xml',
  u'Manchester/anne/anne08a.xml',
  u'Manchester/anne/anne08b.xml',
  u'Manchester/anne/anne09a.xml',
  u'Manchester/anne/anne09b.xml',
  u'Manchester/anne/anne10a.xml',
  u'Manchester/anne/anne10b.xml',
  u'Manchester/anne/anne11a.xml',
  u'Manchester/anne/anne11b.xml',
  u'Manchester/anne/anne12a.xml',
  u'Manchester/anne/anne12b.xml',
  u'Manchester/anne/anne13a.xml',
  u'Manchester/anne/anne13b.xml',
  u'Manchester/anne/anne14a.xml',
  u'Manchester/anne/anne14b.xml',
  u'Manchester/anne/anne15a.xml',
  u'M

In [71]:
def get_mean_length_utterances(file_name):
    # Syntactic Diversity
    child_sent = eng.sents(file_name, speaker="CHI",replace=True)
    for i in range(len(child_sent)):
        child_sent[i] = len(child_sent[i])
    Schild = np.mean(child_sent)

    mother_sent = eng.sents(file_name, speaker="MOT",replace=True)
    for i in range(len(mother_sent)):
        mother_sent[i] = len(mother_sent[i])
    Smother = np.mean(mother_sent)
    return Schild, Smother

In [75]:
def make_freq_dist_files(window_files):
#  Takes a list, where every item is a file
# return a Freq dist for the both the child and mother
    child_words = []
    mother_words = []
    for window_file in window_files:
        child_words += eng.words(window_file,speaker="CHI",replace=True)
        mother_words += eng.words(window_file,speaker="MOT",replace=True)
    return FreqDist(child_words), FreqDist(mother_words)

def make_freq_dist_files_stem(window_files):
#  Takes a list, where every item is a file
# return a Freq dist for the both the child and mother stemmed
    child_words_stem = []
    mother_words_stem = []
    for window_file in window_files:
        child_words_stem += [w.split("-")[0] for w in eng.words(f,speaker="CHI",replace=True,stem=True)]
        mother_words_stem += [w.split("-")[0] for w in eng.words(f,speaker="MOT",replace=True,stem=True)]
    return FreqDist(child_words_stem), FreqDist(mother_words_stem)

def make_windows(child, window_size):
    list_windows = []
    if len(resDict[child]) < window_size:
#         Not Allowed!!
        return False
    else:
        i = 0
        while(i + window_size < len(resDict[child])):
            list_windows.append(resDict[child][i:i+window_size])
            i += 1
    return list_windows

In [ ]:
for child in resDict.keys():
    for window in make_windows(child, 3):
        fchild, fmother = make_freq_dist_files(window)
        fchildS, fmotherS = make_freq_dist_files_stem(window)

        # Statistics
        nchild = sum(array(fchild.values()))
        nmother = sum(array(fmother.values()))
        # Entropies
        Hchild = Ent.Entropy(fchild,method="CWJ")
        Hmother = Ent.Entropy(fmother,method="CWJ")
        # Entropies (stemmed - Lexical Diversity)
        HchildS = Ent.Entropy(fchildS,method="CWJ")
        HmotherS = Ent.Entropy(fmotherS,method="CWJ")
        # Inflectional Diversity
        HchildI = Hchild - HchildS
        HmotherI = Hmother - HmotherS
        
        Schild, Smother = get_mean_length_utterances(f)
        
        print child,age,nchild,Hchild,HchildS,HchildI,Schild,nmother,Hmother,HmotherS,HmotherI,Smother
        print >> fout, child,age,nchild,Hchild,HchildS,HchildI,Schild,nmother,Hmother,HmotherS,HmotherI,Smother

becky 792 1477 4.39223017406 3.93256721805 0.45966295601 1.5 4839 5.40253637561 4.50159337076 0.900943004844 3.54744525547
becky 792 1436 4.39043183862 3.93256721805 0.457864620571 1.5 5064 5.40682183469 4.50159337076 0.905228463931 3.54744525547
becky 792 1356 4.41180047017 3.93256721805 0.479233252121 1.5 5665 5.44786734203 4.50159337076 0.94627397127 3.54744525547
becky 792 1535 4.36700439196 3.93256721805 0.434437173913 1.5 5633 5.4404164652 4.50159337076 0.938823094433 3.54744525547
becky 792 1466 4.47403395454 3.93256721805 0.541466736491 1.5 5993 5.4545930869 4.50159337076 0.952999716141 3.54744525547
becky 792 1616 4.53866566749 3.93256721805 0.606098449442 1.5 5439 5.44866271844 4.50159337076 0.947069347682 3.54744525547
becky 792 1602 4.76480674283 3.93256721805 0.832239524788 1.5 5728 5.49622267003 4.50159337076 0.994629299265 3.54744525547
becky 792 1722 4.68788069439 3.93256721805 0.755313476347 1.5 5411 5.46394220027 4.50159337076 0.962348829511 3.54744525547
becky 792 17

In [25]:
fout = open(nmmfile,"w")
print >> fout, "Child Age N.child H.child H.child.S H.child.I S.child N.mother H.mother H.mother.S H.mother.I S.mother" 
for child,age in resDict.keys():
    ff = resDict[child,age]
    for f in ff:
        # Frequency dists of child and mother for this particular child and age (raw) 
        # Bottleneck of runtime
        print type(eng.words(f,speaker="CHI",replace=True))
        fchild = FreqDist(eng.words(f,speaker="CHI",replace=True))
        fmother = FreqDist(eng.words(f,speaker="MOT",replace=True))
        # Frequency dists of child and mother for this particular child and age (stemmed) 
        # Notice that we split words on "-" to transform "be-3S" to "be"
        # Bottleneck of runtime
        
#         fchildS = FreqDist([w.split("-")[0] for w in eng.words(f,speaker="CHI",replace=True,stem=True)])
#         fmotherS = FreqDist([w.split("-")[0] for w in eng.words(f,speaker="MOT",replace=True,stem=True)])
#         # Statistics
#         nchild = sum(array(fchild.values()))
#         nmother = sum(array(fmother.values()))
#         # Entropies
#         Hchild = Ent.Entropy(fchild,method="CWJ")
#         Hmother = Ent.Entropy(fmother,method="CWJ")
#         # Entropies (stemmed - Lexical Diversity)
#         HchildS = Ent.Entropy(fchildS,method="CWJ")
#         HmotherS = Ent.Entropy(fmotherS,method="CWJ")
#         # Inflectional Diversity
#         HchildI = Hchild - HchildS
#         HmotherI = Hmother - HmotherS
        
#         Schild, Smother = get_mean_length_utterances(f)
        
#         print child,age,nchild,Hchild,HchildS,HchildI,Schild,nmother,Hmother,HmotherS,HmotherI,Smother
#         print >> fout, child,age,nchild,Hchild,HchildS,HchildI,Schild,nmother,Hmother,HmotherS,HmotherI,Smother

<type 'list'>
<type 'list'>
<type 'list'>
<type 'list'>
<type 'list'>
<type 'list'>


KeyboardInterrupt: 

[u'Manchester/anne/anne01a.xml',
 u'Manchester/anne/anne01b.xml',
 u'Manchester/anne/anne02a.xml',
 u'Manchester/anne/anne02b.xml',
 u'Manchester/anne/anne03a.xml',
 u'Manchester/anne/anne03b.xml',
 u'Manchester/anne/anne04a.xml',
 u'Manchester/anne/anne04b.xml',
 u'Manchester/anne/anne05a.xml',
 u'Manchester/anne/anne05b.xml',
 u'Manchester/anne/anne06a.xml',
 u'Manchester/anne/anne06b.xml',
 u'Manchester/anne/anne07a.xml',
 u'Manchester/anne/anne07b.xml',
 u'Manchester/anne/anne08a.xml',
 u'Manchester/anne/anne08b.xml',
 u'Manchester/anne/anne09a.xml',
 u'Manchester/anne/anne09b.xml',
 u'Manchester/anne/anne10a.xml',
 u'Manchester/anne/anne10b.xml',
 u'Manchester/anne/anne11a.xml',
 u'Manchester/anne/anne11b.xml',
 u'Manchester/anne/anne12a.xml',
 u'Manchester/anne/anne12b.xml',
 u'Manchester/anne/anne13a.xml',
 u'Manchester/anne/anne13b.xml',
 u'Manchester/anne/anne14a.xml',
 u'Manchester/anne/anne14b.xml',
 u'Manchester/anne/anne15a.xml',
 u'Manchester/anne/anne15b.xml',
 u'Manches